# Análisis de puestos

In [133]:
import pandas as pd
import pandas_profiling
import numpy as np
from pandasql import *

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [134]:
df = pd.read_pickle('sueldos_contrata.pkl')

In [135]:
pandas_profiling.ProfileReport(df)

Number of variables,8
Number of observations,16241
Total Missing (%),0.0%
Total size in memory,1015.1 KiB
Average record size in memory,64.0 B
Numeric,2
Categorical,6
Date,0
Text (Unique),0
Rejected,0
Distinct count,2305


## Limpieza de variable Estamento

Veamos cómo es la distribución de valores de esta variable

In [136]:
q  = "SELECT Estamento,count(*) as Nro FROM df GROUP BY Estamento ORDER BY 2 desc"
sqldf(q)

,Estamento,Nro
0,PROFESIONAL,9570
1,Profesional,4012
2,Técnico,527
3,ADMINISTRATIVO,472
4,Administrativo,463
5,d. Profesional,231
6,TÉCNICO,189
7,DIRECTIVO,149
8,FISCALIZADORES,100
9,TECNICO,76


Observaciones:

* Mezcla de mayúsculas y minúsculas
* Abreviaciones con puntuaciones extrañas (d.,j.,h.,g.)
* Existe el concepto "No Aplica". Hay que investigar la razón.

Para la limpieza, vamos a eliminar todas las puntuaciones y reemplazaré algunas abreviaciones. Examinemos primero estos casos raros.

In [137]:
q  = '''
SELECT 
    distinct Estamento,
    Cargo, 
    Servicio 
FROM 
    df 
WHERE 
    Estamento like "_.%"
'''
sqldf(q)

,Estamento,Cargo,servicio
0,d. Profesional,PROFESIONAL,Servicio Evaluación Ambiental
1,d. Profesional,JEFA DE DEPARTAMENTO,Servicio Evaluación Ambiental
2,g. Tco No Profes,ADMINISTRATIVO,Servicio Evaluación Ambiental
3,h. Admistrativo,ADMINISTRATIVO,Servicio Evaluación Ambiental
4,g. Tco No Profes,TECNICO,Servicio Evaluación Ambiental
5,j. Auxiliar,AUXILIAR,Servicio Evaluación Ambiental
6,d. Profesional,JEFE DE DEPARTAMENTO,Servicio Evaluación Ambiental
7,h. Admistrativo,TECNICO,Servicio Evaluación Ambiental


Observamos que todos pertencen al Servicio de Evaluación Ambiental. Si miramos la fuente, podemos ver que efectivamente no hay referencias del significado de los prefijos mostrados, por lo que se eliminarán.

In [138]:
# No hay stop_words a borrar. Normalmente se usaría stop_words = stopwords.words('spanish')
stop_words = []

In [139]:
# Borramos abreviaciones sin significado
palabras_a_borrar = ['d.','g.','h.','j.']

# Normalmente se usa puntuaciones = string.punctuation
puntuaciones = []

In [140]:
def clean_text(s):
    
    # 1. Pasar todo a minúsculas y separar en arreglo de palabras
    ws = s.lower().split()

    # 2. Eliminar espaciones innecesarios, junto con conectores gramaticales'''
    ws = [w.strip() for w in ws if w not in stop_words]
    
    # 3. Remover puntuación
    ws = [w for w in ws if w not in puntuaciones]

    # 4. 
    ws = [w for w in ws if w not in palabras_a_borrar]
    return " ".join(ws)

In [141]:
estamentos = df['Estamento'].tolist()

In [142]:
estamentos = pd.Series([clean_text(s) for s in estamentos])

In [143]:
# Uniformemos valores con un diccionario y reemplazamos
d = {
    'técnico': 'tecnico',
    'fiscalizadores': 'fiscalizador',
    'administrativos': 'administrativo',
    'profesionales': 'profesional',
    'tco no profes': 'tecnico',
    'no aplica': 'otro',
    'auxiliares': 'auxiliar',
    'técnicos': 'tecnico'
}
estamentos.replace(d,inplace=True)

In [145]:
estamentos.sort_values().value_counts()

profesional       13885
administrativo      959
tecnico             835
fiscalizador        201
directivo           164
auxiliar            128
admistrativo         27
otro                 26
experto              16
dtype: int64

No aparecen más problemas.

## Limpieza de variable Cargo

In [19]:
# Primero veamos la lista de cargos con mayor frecuencia de aparición, ej. con frecuencia > 50
q  = "SELECT Cargo,count(*) as Nro FROM df GROUP BY Cargo HAVING Nro>50 ORDER BY 2 desc"
sqldf(q)

,Cargo,Nro
0,PROFESIONAL,788
1,PROFESIONAL SECTORIAL,572
2,PROFESIONAL DIRECCION REGIONAL,494
3,PROFESIONAL DIVISION TECNICA,390
4,JEFE SUBDEPARTAMENTO TECNICO,286
5,PROFESIONAL DEPARTAMENTO DE SOPORTE,234
6,Profesional Analista,234
7,JEFE SUBDEPARTAMENTO DE SOPORTE,208
8,31/12/2016,188
9,PROFESIONAL DE APOYO,118


Podemos observar que hay fechas metidas entre medio. Veamos los casos:

In [20]:
s = 'select servicio,count(*) as nro from df where Cargo like "%/2016" group by Cargo'
sqldf(s)

,servicio,nro
0,Superindencia del Medio Ambiente,2
1,Superindencia del Medio Ambiente,2
2,Superindencia del Medio Ambiente,2
3,Superindencia del Medio Ambiente,2
4,Superindencia del Medio Ambiente,4
5,Superindencia del Medio Ambiente,188


Nos encontramos con que los casos corresponden a la misma institución (qu.e además tiene el nombre malo). Revisando la página web, podemos ver que el problema es del origen:

![](error1.png)

Por lo que borraremos estos registros en el merge final

In [147]:
# Pasamos los cargos a una variable para limpiar
cargos = df['Cargo'].tolist()

In [148]:
# Para usar la función "clean_text" usaremos además limpieza de conectores gramaticales de la lengua española
stop_words = stopwords.words('spanish')
palabras_a_borrar = []
puntuaciones = string.punctuation

In [149]:
cargos = pd.Series([clean_text(s) for s in cargos])

In [153]:
# Muestra para ver como quedó
cargos[0:10].to_frame(name='cargo')

,cargo
0,jefa unidad comunicaciones
1,asesor vicepresidencia
2,jefa auditoria interna
3,jefe depto. contabilidad
4,asesor legal cds r
5,jefe departamento fondo solidiario
6,jefa subrogante departamento pensiones
7,abogado garantía hipotecaria
8,jefa unidad capacitación evaluación desempeño
9,encargado departamento asistencia social


In [154]:
# Veamos una lista general
df_cargos = cargos.to_frame(name='cargo')
q  = "SELECT cargo,count(*) as nro FROM df_cargos GROUP BY cargo ORDER BY 2 desc"
sqldf(q)

,cargo,nro
0,profesional,794
1,profesional sectorial,572
2,profesional direccion regional,495
3,profesional division tecnica,390
4,jefe subdepartamento tecnico,286
5,profesional analista,234
6,profesional departamento soporte,234
7,jefe subdepartamento soporte,208
8,31/12/2016,188
9,profesional apoyo,174


Para hacer la limpieza más eficiente, vamos a separar el nombre del cargo en dos componentes: 

* La denominación principal del cargo, ej.: jefe, encargado, tecnico
* Área de aplicación

Luego procederemos a normalizar cada campo:

* Unificando género, ej.: jefe = jefa
* Eliminando subrogancias
* Limpiando acrónimos, ej: cdr r
* Normalizando textos equivalentes en significado: depto. = departamento 

In [155]:
# Creamos un nuevo Data Frame expandiendo por el primer split. Luego de las correciones los uniremos
df_cargos = df_cargos['cargo'].str.split(' ', 1, expand=True)
df_cargos.columns = ['nivel','ambito']
df_cargos.head()

,nivel,ambito
0,jefa,unidad comunicaciones
1,asesor,vicepresidencia
2,jefa,auditoria interna
3,jefe,depto. contabilidad
4,asesor,legal cds r


In [156]:
# Veamos las correcciones al campo "nivel"
s='select nivel,count(*) as nro from df_cargos group by nivel having nro>1 order by 2 desc'
sqldf(s)

,nivel,nro
0,profesional,5276
1,jefe,2104
2,analista,1341
3,encargado,583
4,jefa,498
5,ejecutivo,454
6,abogado,451
7,asistente,427
8,encargada,421
9,administrativo,398


La lista nos muestra varios tipos de reemplazos. Haremos un diccionario y aplicaremos las correcciones

In [157]:
# Cambiemos acentos
# Uniformemos valores con un diccionario y reemplazamos (sugerencia: cargar desde archivo dict.txt)
d = {
    'director/a':'director',
    'directora':'director',
    'subdirector/a': 'subdirector',
    'subdirectora': 'subdirector',
    'jefa':'jefe',
    'jefe/a':'jefe',
    'jefe(a)':'jefe',
    'jefatura': 'jefe',
    'jefatura,': 'jefe',
    'consultor/a,': 'consultor',
    'consultora': 'consultor',
    'asesora': 'asesor',
    'asesor(a)': 'asesor',
    'asesoria': 'asesor',
    'asesora,':'asesor',
    'enc.': 'encargado',
    'encargada':'encargado',
    'encargada,': 'encargado',
    'encargada(s)':'encargado',
    'encargado/a': 'encargado',
    'encargado(a)': 'encargado',
    'encagado': 'encargado',
    'coordinador(a)': 'coordinador',
    'coordinadora': 'coordinador',
    'coordinador/a': 'coordinador',
    'abogado/a': 'abogado',
    'técnico': 'tecnico',
    'tecnico(a)': 'tecnico',
    'técnico(a)' : 'tecnico',
    'estadístico': 'estadistico',
    'fiscalizadora': 'fiscalizador',
    'ejecutiva':'ejecutivo',
    'operadora': 'operador',
    'abogada': 'abogado',
    'profesionalde': 'profesional',
    'jede': 'jefe',
    'tesorera': 'tesorero',
    'adm.': 'administrativo',
    'administrativo/a': 'administrativo',
    'administrativa': 'administrativo',
    'administrativo(a)': 'administrativo',
    'administrtivo' : 'administrativo',
    'analista,': 'analista',
    'editora': 'editor',
    'guía': 'guia',
    'digitadora': 'digitador',
    'examinadora': 'examinador',
    'secretaria,': 'secretaria',
    'transcriptor,': 'transcriptor',
    'auditora': 'auditor',
    'profesional,' : 'profesional',
    'profsional' : 'profesional',
    'prosefional' : 'profesional',
    'reportera' : 'reportero',
    'nutricionista,' : 'nutricionista',
    'profecional' : 'profesional',
    'restaurador,' : 'restaurador',
    'psicólogo' : 'psicologo',
    'trabajadora': 'trabajador',
    'asistentente': 'asistente',
    'supervisora': 'supervisor',
    'líder' : 'lider',
    'secretario' : 'secretariado',
    'secretaria' : 'secretariado',
    'admninistrativo' : 'administrativo',
    'administración': 'administrativo',
    'experta': 'experto'
}
df_cargos["nivel"].replace(d,inplace=True)

In [158]:
s = 'select nivel, count(*) as nro from df_cargos group by nivel having nro>1 order by 2 desc'
print(sqldf(s)['nivel'].tolist())

['profesional', 'jefe', 'analista', 'encargado', 'abogado', 'administrativo', 'ejecutivo', 'asistente', 'coordinador', 'secretariado', 'tecnico', 'asesor', 'apoyo', 'director', '31/12/2016', 'auditor', 'agregado', 'auxiliar', 'fiscalizador', 'conductor', 'ingeniero', 'periodista', 'economista', 'consultor/a', 'agente', 'prevencionista', 'subdirector', 'chofer', 'estadistico', 'oirs', 'sociologo', 'supervisor', 'actuario', 'examinador', 'intendente', 'editor', 'guia', 'receptor', 'tesorero', 'geografo', 'subjefe', 'consultor', 'operador', 'chef', 'oficial', 'reportero', 'control', 'administrador', 'contador', 'contralor', 'desarrollar', 'digitador', 'experto', 'soporte', 'asist.', 'psicologo', 'revisor', 'seremi', '31/01/2016', 'coordinar', 'daf', 'desarrollador', 'electricista,', 'estafeta', 'estafeta,', 'gasfiter', 'maestro', 'nutricionista', 'programa', 'representante', 'restaurador', 'sectorialista', 'trabajador', 'transcriptor', 'analisis', 'delegado', 'dpto.', 'gabinete', 'gestor'

Revisaremos además:

* asist.
* sub
* analisis
* dpto.
* gabinete
* desarrollar
* coordinar
* programa
* secretaría
* oficina

In [159]:
s = '''
select * from df where Cargo like "asist. %" union
select * from df where Cargo like "sub %" union
select * from df where Cargo like "analisis %" union
select * from df where Cargo like "dpto. %" union
select * from df where Cargo like "gabinete %" union
select * from df where Cargo like "desarrollar %" union
select * from df where Cargo like "programa %" union
select * from df where Cargo like "secretaría %" union
select * from df where Cargo like "oficina %"
'''
sqldf(s)

,Estamento,Cargo,Grado EUS,Remuneración Bruta Mensualizada,Fecha de inicio,Fecha de término,servicio
0,ADMINISTRATIVO,PROGRAMA MUJER Y TRABAJO,12,1657601.0,01/01/2016,31/12/2016,Servicio Nacional de la Mujer (SERNAM)
1,ADMINISTRATIVO,Secretaría Apoyo Asesores,18,807364.0,01/01/2016,31/12/2016,Subsecretaría de Hacienda
2,Auxiliar,OFICINA DE PARTES,17,1074499.0,01/03/2016,31/10/2016,Gobierno Regional de Aysén
3,PROFESIONAL,"ANALISIS DE CUENTA, CONCILIACIONES BANCARIAS Y...",19,842932.0,16/03/2016,31/12/2016,Subsecretaría de Economía
4,PROFESIONAL,DESARROLLAR PROYECTO TRIBUNAL SIN PAPEL. ESTUD...,10,1831080.0,02/05/2016,08/08/2016,Subsecretaría de Economía
5,PROFESIONAL,DESARROLLAR PROYECTO TRIBUNAL SIN PAPEL. ESTUD...,10,1831080.0,21/03/2016,01/05/2016,Subsecretaría de Economía
6,PROFESIONAL,Dpto. Técnico de Sistemas de Combustibles,08º,3356200.0,24/06/2011,31/12/2016,Superintendencia de Electricidad y Combustible...
7,PROFESIONALES,Sub Jefe División Abusos Unilaterales,4,5197469.0,01/01/2016,31/12/2016,Fiscalía Nacional Económica (FNE)
8,Profesional,Gabinete Subsecretaria,11° EUS,1629768.0,01/01/2016,31/12/2016,Subsecretaría de Energía
9,Técnico,Asist. Tec. Servicios Generales,18,602428.0,01/01/2016,31/12/2016,SERVIU IX Región


Hay varias situaciones:

* Mezclas de cargos con función
* Se escribe nombre de área de impacto
* asist. = asistente
* sub = subjefe

En los primeros dos casos, reemplazaremos el cargo por el nombre del estamento

In [160]:
d = {}
d = {
    'asist.': 'asistente',
    'programa': 'administrativo', 
    'secretaría' : 'administrativo',
    'analisis': 'profesional',
    'dpto.': 'profesional',
    'gabinete': 'profesional',
    'desarrollar': 'profesional',
    'coordinar': 'profesional',
    'programa': 'profesional',
    'sub': 'subjefe',
    'oficina': 'auxiliar'
}
df_cargos["nivel"].replace(d,inplace=True)

In [161]:
# Veamos ahora los cargos de menor frecuencia
s='select nivel,count(*) as nro from df_cargos group by nivel having nro=1 order by 2 desc'
sqldf(s)

,nivel,nro
0,&nbs,1
1,administrativo-chofer,1
2,administrativo-estafeta,1
3,administrativo-servicios,1
4,adquisiciones,1
5,apoyotécnico,1
6,comunicaciones,1
7,contabilidad,1
8,contralora,1
9,enc.unidad,1


In [162]:
d = {
    'contralora': 'contralor',
    'administrativo-chofer' : 'chofer',
    'administrativo-estafeta' : 'estafeta',
    'administrativo-servicios' : 'admninistrativo',
    'enc.unidad' : 'encargado',
    'apoyotécnico' : 'técnico',
}
df_cargos["nivel"].replace(d,inplace=True)

In [163]:
# Veamos ahora los cargos de menor frecuencia
s='select * from df where Cargo like "&nbs%"'
sqldf(s)

,Estamento,Cargo,Grado EUS,Remuneración Bruta Mensualizada,Fecha de inicio,Fecha de término,servicio
0,Profesional,&nbs,5,3216550.0,01/01/2011,31/12/2016,SERVIU Arica y Parinacota


Cambiamos por el estamento

In [164]:
d = {'&nbs': 'profesional'}
df_cargos["nivel"].replace(d,inplace=True)

In [165]:
# Hagamos una revisión
print(df_cargos['nivel'].unique().tolist())

['jefe', 'asesor', 'abogado', 'encargado', 'agente', 'profesional', 'analista', 'director', 'operador', 'asistente', 'consultor', 'psicologo', 'administrador', 'tesorero', 'auditor', 'adquisiciones', 'contabilidad', 'ejecutivo', 'estafeta', 'chofer', 'admninistrativo', 'administrativo', 'secretariado', 'apoyo', 'coordinador', 'fiscal', 'ingeniero', 'lider', 'desarrollador', 'auxiliar', 'tecnico', 'consultor/a', 'subdirector', 'economista', 'subjefe', 'contralor', 'periodista', 'egresado', 'oficial', 'conductor', 'contador', 'técnico', 'servicios', 'mayordomo', 'prevencionista', 'revisor', 'sociologo', 'estadistico', 'geografo', 'examinador', 'soporte', 'digitador', 'reportero', 'representante', 'nutricionista', 'chef', 'transcriptor', 'guia', 'maestro', 'restaurador', 'electricista,', 'estafeta,', 'oirs', 'sectorialista', 'comunicaciones', 'delegado', 'receptor', 'experto', 'portero', 'gestor', 'fiscalizador', 'control', 'agregado', 'seremi', 'editor', 'trabajador', 'gasfiter', 'daf', 

Pareciera estar bastante mejor. Veamos ahora el ámbito

In [166]:
s = 'select distinct ambito from df_cargos order by 1 asc'
sqldf(s)

,ambito
0,None
1,(a) capacitación- recursos humanos
2,(a) catastro
3,"(a) cocina, departamento repostero presidencial"
4,(a) departamento estudios economicos coyunturales
5,(a) departamento estudios laborales
6,(a) departamento gestion financiera
7,(a) departamento judicial
8,(a) gabinete ministro (a) justicia
9,(a) gabinete subsecretario (a) justicia


In [44]:
rep

,nivel
profesional,33.133845
jefe,17.642292
analista,8.509445
encargado,7.406022
abogado,3.378842
administrativo,2.911290
ejecutivo,2.842716
asistente,2.711801
coordinador,2.462440
secretariado,2.331525


## Limpieza de variable Grado

In [48]:
s = 'select distinct `Grado EUS` from df'
print(sqldf(s)['Grado EUS'].tolist())

['4°', '6°', '7°', '8°', '9°', '10°', 'I', 'II', 'III', '7', '14', '16', '6', '8', '9', '10', '11', '4', '5', '12', '13', '2', '5°', '11°', '12°', '13°', '14°', '20°', '22', '3', '15', '1C', '17', '19', '20', '21', '18', '06', '05', '07', '09', '08', '04', '9° EUS', '6° EUS', '4° EUS', '7° EUS', '11° EUS', '8° EUS', '10° EUS', '2° EUS', '5° EUS', '13° EUS', '16° EUS', '12° EUS', 'Región Metropolitana de Santiago', 'Región del Biobío', 'Atacama', 'Región Aisén del Gral. Carlos Ibáñez del Campo', 'Región del Libertador Gral. Bernardo OHiggins', 'Región de Los Lagos', 'Región de la Araucanía', 'Región de Los Ríos', 'Tarapacá', '04º', '05º', '06º', '08º', '09º', '10º', '11º', '12º']


Hay varias correcciones que hacer:

* Sacar el símbolo de grado (º)
* Eliminar la palabra EUS
* Examinar aquellos registros en que aparece una región

In [76]:
# Limpieza de símbolos
grados = df['Grado EUS']
grados = grados.str.replace('°','')
grados = grados.str.replace('º','')
grados = grados.str.replace(' EUS','')
grados = grados.str.replace('1C','1')

In [77]:
print(grados.unique().tolist())

['4', '6', '7', '8', '9', '10', 'I', 'II', 'III', '14', '16', '11', '5', '12', '13', '2', '20', '22', '3', '15', '1', '17', '19', '21', '18', '06', '05', '07', '09', '08', '04', 'Región Metropolitana de Santiago', 'Región del Biobío', 'Atacama', 'Región Aisén del Gral. Carlos Ibáñez del Campo', 'Región del Libertador Gral. Bernardo OHiggins', 'Región de Los Lagos', 'Región de la Araucanía', 'Región de Los Ríos', 'Tarapacá']


Examinemos los casos anómalos

In [74]:
s = 'select servicio,count(*) as nro from df where `Grado EUS` like "Reg%" group by servicio'
sqldf(s)

,servicio,nro
0,Subsecretaría del Medio Ambiente,485


Todos los datos pertenecen a esta institución. El error viene de la fuente:

![](error2.png)

Como tenemos la Remuneración Mensualizada Bruta, podemos imputar el grado con el valor de la mediana de asociada a cada grado, ya que la información de grados de cada página es imprecisa para hacer esta imputación

In [81]:
# Reemplazaremos los datos anómalos por Nulo e imputaremos posteriormente
d = {
    'I' : '0', 
    'II' : '0', 
    'III' : '0',
    'Región Metropolitana de Santiago' : '0', 
    'Región del Biobío' : '0', 
    'Atacama' : '0', 
    'Región Aisén del Gral. Carlos Ibáñez del Campo' : '0', 
    'Región del Libertador Gral. Bernardo OHiggins' : '0', 
    'Región de Los Lagos' : '0', 
    'Región de la Araucanía' : '0', 
    'Región de Los Ríos' : '0', 
    'Tarapacá' : '0'
}
grados.replace(d,inplace=True)
grados = grados.astype('int')
print(grados.unique().tolist())

[4, 6, 7, 8, 9, 10, 0, 14, 16, 11, 5, 12, 13, 2, 20, 22, 3, 15, 1, 17, 19, 21, 18]


## Limpieza de variables fecha de inicio y término

In [202]:
s = 'select distinct `Fecha de inicio` from df'
print(sqldf(s)['Fecha de inicio'].tolist())

['01/08/2014', '01/03/2015', '22/12/2011', '26/05/2014', '02/01/1996', '01/11/2004', '20/09/1995', '15/10/1999', '24/05/2010', '01/06/2014', '13/12/2010', '07/11/1994', '28/07/2014', '23/07/2012', '27/01/1995', '01/12/1999', '18/08/2014', '01/03/2011', '01/06/2012', '09/09/2012', '01/01/2009', '01/01/1995', '18/04/2016', '01/06/1991', '01/01/2005', '01/02/2014', '10/01/2000', '01/07/2001', '01/07/1995', '01/02/2000', '01/08/2011', '01/02/2013', '01/01/1988', '01/09/1986', '01/08/2001', '23/12/2013', '01/11/1997', '15/12/2014', '01/01/2003', '01/12/1989', '01/10/2007', '16/01/2012', '01/01/2000', '09/06/2014', '06/04/1998', '04/10/2007', '01/09/2006', '01/06/1998', '02/11/2001', '01/03/1981', '28/05/1996', '08/07/2008', '20/10/1999', '15/11/2004', '01/10/1994', '07/06/1999', '23/10/1995', '13/11/1995', '25/08/1989', '01/07/2006', '01/10/1989', '01/02/1994', '01/01/2016', '15/04/1994', '02/10/1995', '11/07/2012', '01/11/2003', '01/06/1994', '01/10/2006', '01/12/1991', '01/08/2010', '01/1

Un examen preliminar de los datos no evidencia elementos extraños en el campo. Intentaremos convertir a tipo fecha.

In [203]:
fecha_inicio = df['Fecha de inicio']

In [204]:
df.columns

Index(['Estamento', 'Cargo', 'Grado EUS', 'Remuneración Bruta Mensualizada',
       'Fecha de inicio', 'Fecha de término', 'servicio'],
      dtype='object')

In [205]:
s = 'select distinct `Fecha de término` from df'
print(sqldf(s)['Fecha de término'].tolist())

['31/12/2016', '05/06/2016', '31/05/2016', '18/03/2016', '06/02/2016', '18/10/2016', '31/01/2016', '04/05/2016', '30/06/2016', '17/03/2016', '29/02/2016', '17/07/2016', '05/04/2016', '27/06/2016', '25/02/2016', '31/07/2016', '21/08/2016', '05/02/2016', '11/08/2016', '16/08/2016', '31/03/2016', '18/08/2016', '26/07/2016', '12/10/2016', '20/03/2016', '26/08/2016', '06/03/2016', '30/04/2016', '15/04/2016', '19/06/2016', '15/08/2016', '14/04/2016', '19/07/2016', '08/05/2016', 'Indefinido', '06/07/2016', '30/09/2015', '12/07/2016', '17/04/2016', '19/02/2016', '08/08/2016', '10/03/2016', '22/03/2016', '27/03/2016', '15/03/2016', '17/01/2016', '', '31/10/2016', '30/09/2016', '12/06/2016', '21/07/2016', '13/08/2016', '21/06/2016', '30/08/2016', '05/03/2016', '03/07/2016', '19/05/2016', '01/08/2016', '27/02/2016', '23/05/2016', '18/04/2016', '22/06/2016', '15/05/2016', '03/02/2016', '13/10/2016', '22/07/2016', '01/05/2016', '18/05/2016', '03/10/2016', '28/08/2016', '19/08/2016', '21/05/2016', '

En el caso de la fecha de término, observamos que la fecha de término contiene el string "Indefinido", y en ambos campos no hay valores nulos. Reemplazaremos Indefinido por nulo.

In [207]:
# Reemplazo de indefinido por nulo
fecha_termino = df['Fecha de término'].replace('Indefinido',np.nan)

No hay más correcciones. Ensamblaremos el Data Frame final.

In [259]:
# Data Frame final
datos = {
    'estamento': estamentos,
    'nivel': df_cargos['nivel'].tolist(),
    'ambito': df_cargos['ambito'].tolist(),
    'grado': grados.values,
    'rbm': df['Remuneración Bruta Mensualizada'].tolist(),
    'fecha_inicio': fecha_inicio.values,
    'fecha_termino': fecha_termino.values,
    'servicio': df['servicio'].values
}
df_final = pd.DataFrame(datos)
df_final.head()

,ambito,estamento,fecha_inicio,fecha_termino,grado,nivel,rbm,servicio
0,unidad comunicaciones,profesional,01/08/2014,31/12/2016,4,jefe,3639594.0,Caja de Previsión de la Defensa Nacional (CAPR...
1,vicepresidencia,profesional,01/03/2015,05/06/2016,4,asesor,3900396.0,Caja de Previsión de la Defensa Nacional (CAPR...
2,auditoria interna,profesional,22/12/2011,31/12/2016,4,jefe,2911675.0,Caja de Previsión de la Defensa Nacional (CAPR...
3,depto. contabilidad,profesional,26/05/2014,31/12/2016,6,jefe,2379684.0,Caja de Previsión de la Defensa Nacional (CAPR...
4,legal cds r,profesional,02/01/1996,31/12/2016,7,asesor,2393036.0,Caja de Previsión de la Defensa Nacional (CAPR...


In [260]:
# Cambiamos el order
cols = ['servicio','estamento','grado','nivel','ambito','fecha_inicio','fecha_termino','rbm']
df_final = df_final[cols]
df_final.head()

,servicio,estamento,grado,nivel,ambito,fecha_inicio,fecha_termino,rbm
0,Caja de Previsión de la Defensa Nacional (CAPR...,profesional,4,jefe,unidad comunicaciones,01/08/2014,31/12/2016,3639594.0
1,Caja de Previsión de la Defensa Nacional (CAPR...,profesional,4,asesor,vicepresidencia,01/03/2015,05/06/2016,3900396.0
2,Caja de Previsión de la Defensa Nacional (CAPR...,profesional,4,jefe,auditoria interna,22/12/2011,31/12/2016,2911675.0
3,Caja de Previsión de la Defensa Nacional (CAPR...,profesional,6,jefe,depto. contabilidad,26/05/2014,31/12/2016,2379684.0
4,Caja de Previsión de la Defensa Nacional (CAPR...,profesional,7,asesor,legal cds r,02/01/1996,31/12/2016,2393036.0


In [261]:
s = 'select * from df_final where nivel not like "%/2016"'
df_final = sqldf(s)

In [262]:
df_final['fecha_inicio'] = pd.to_datetime(df_final['fecha_inicio'])
df_final['fecha_termino'] = pd.to_datetime(df_final['fecha_termino'])

In [263]:
df_final.to_pickle('sueldos_contrata_clean.pkl')